# Solving a SPP instance with a MIP solver

The purpose of this tutorial is to demonstrate how to read an instance of Set Packing Problem.

---- 
## Loading a SPP instance

#### Load the parser

In [ ]:
path = "../2.codes/"

In [ ]:
include(path * "MICparse.jl")

#### Read an instance:

In [ ]:
fname = "didactic.dat"
C, A  = loadSPP(fname)
m,n   = size(A)

---- 

## JuMP formulation

This tutorial requires the following packages:

In [ ]:
using JuMP
using HiGHS

First, we'll create a `Model` object for holding model elements as we
construct each part. We'll also set the solver that will ultimately be called
to solve the model, once it's constructed.

In [ ]:
spp = Model(HiGHS.Optimizer)

Next we declare (1) the decision variables:

In [ ]:
@variable(spp, x[1:n], Bin)

(2) the constraints:

In [ ]:
@constraint(spp, cst[i=1:m], sum(A[i,j]*x[j] for j=1:n) ≤ 1)

and finally, (3) the objective function:

In [ ]:
@objective(spp, Max, sum(C[j]*x[j] for j=1:n))

Let's print a human-readable description of the model and check that the model
looks as expected:

In [ ]:
print(spp)

We can now solve the optimization problem and inspect the results.

In [ ]:
optimize!(spp)
solution_summary(spp)

The results are:

In [ ]:
@show objective_value(spp)

In [ ]:
@show  value(x[2])

In [ ]:
@show  value.(x)

In [ ]:
x1 = [i for i in 1:n if value(x[i]) > 0.5]

----- 

## Writing a function

The function can be used to ensure that the model is given well-defined input
data with validation checks, and that the solution process went as expected.

In [ ]:
function solve_SPP(C::Vector{Int64}, A::Matrix{Int64} )
    
    m,n = size(A)
    @assert length(C) == n

    spp = Model(HiGHS.Optimizer)
    
    set_silent(spp)
    
    @variable(spp, x[1:n], Bin)
    @constraint(spp, cst[i=1:m], sum(A[i,j]*x[j] for j=1:n) ≤ 1)
    @objective(spp, Max, sum(C[j]*x[j] for j=1:n))
    
    optimize!(spp)
    @assert termination_status(spp) == OPTIMAL
    @assert primal_status(spp) == FEASIBLE_POINT
    
    zOpt = objective_value(spp)
    x1Opt = [i for i in 1:n if value(x[i]) > 0.5]

    println("Objective : z=", round(Int,zOpt))
    print("Solution  : ")
    for i in 1:n
        print("x[$i]=", round(Int, value(x[i])), "  ")
    end

    return x1Opt, zOpt
end

In [ ]:
solve_SPP(C,A)